In [4]:

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten, Conv2D, MaxPooling2D

from google.cloud import bigquery

In [5]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "noble-district-459911-t9-1c0ed90c64dd.json"

In [7]:

from google.cloud import bigquery
client = bigquery.Client(project="noble-district-459911-t9")



DefaultCredentialsError: File noble-district-459911-t9-1c0ed90c64dd.json was not found.

In [ ]:
query = """
SELECT
  title,
  answer_count,
  REPLACE(tags, "|", ",") as tags
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
  REGEXP_CONTAINS( tags, r"(?:keras|matplotlib|pandas)")
LIMIT 1000
 """

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:

stacko_vocab_size = 200
stacko_sequence_len = 40

# Create a tokenizer for this data
stacko_tokenize = keras.preprocessing.text.Tokenizer(num_words=stacko_vocab_size)
stacko_tokenize.fit_on_texts(df['title'].values)

In [ ]:

# Preview the first 20 words in the Tokenizer's vocabulary
list(stacko_tokenize.word_index.keys())[:20]

In [ ]:
questions_train_embedding = stacko_tokenize.texts_to_sequences(df['title'].values)
questions_train_embedding = keras.preprocessing.sequence.pad_sequences(questions_train_embedding, maxlen=stacko_sequence_len, padding='post')


In [ ]:

# Preview the embedding representation with the actual input text
print(df['title'].iloc[0])
print(questions_train_embedding[0])

In [ ]:
questions_train_matrix = stacko_tokenize.texts_to_matrix(df['title'].values)
print(questions_train_matrix[0])

In [ ]:

batch_size = len(df) # Note: we're using the whole dataset as the batch size for demo purposes

embedding_input = Input(shape=(stacko_sequence_len,))
embedding_layer = Embedding(batch_size, 64)(embedding_input)
embedding_layer = Flatten()(embedding_layer)
embedding_layer = Dense(32, activation='relu')(embedding_layer)

bow_input = Input(shape=(stacko_vocab_size,))
bow_layer = Dense(32, activation='relu')(bow_input)

In [ ]:
# Try to combine the two inputs